# Enhanced detection of DNS tunnelling: Leveraging random forest and genetic algorithm for improved security

To download the necessary CSV files for the project, use the following `wget` commands:


In [ ]:
import os
import requests

# List of file URLs to download
file_urls = [
    ("https://drive.usercontent.google.com/download?id=1cictwnxUyu1vCa4H9iefIrQeVLCC3RCv&export=download&authuser=0&confirm=t&uuid=8ec5d698-4d5d-4592-94eb-8a82234966ac&at=AC2mKKTzwehwnBUepaEJIDoKDql-:1690876827674", "benign-chrome.csv"),
    ("https://drive.usercontent.google.com/download?id=1cms99qEylyvesqcX3dQRZOUQRAONy2uS&export=download&authuser=0&confirm=t&uuid=0f089685-41f1-40fe-903e-8fcc8e2bcac8&at=AC2mKKSfqH9g0sjW4mQVa5-J4gMf:1690877149684", "benign-firefox.csv"),
    ("https://drive.usercontent.google.com/download?id=1cqDL7A_kdOCL4Km4uUifRPllFmB3WaZ_&export=download&authuser=0&confirm=t&uuid=19171c97-ad00-4af4-bf46-ef8c453b2964&at=AC2mKKROICucTfu1coxAIff16wi1:1690878058234", "mal-dns2tcp.csv"),
    ("https://drive.usercontent.google.com/download?id=1cxeTvXNV-OY_4T6xs4sUB98lmanROw3m&export=download&authuser=0&confirm=t&uuid=67df7c64-15ed-450d-bad8-f416080d378d&at=AC2mKKST9kQGoFcvwe9EhJoY6jRA:1690878087508", "mal-dnscat2.csv"),
    ("https://drive.google.com/u/1/uc?id=1czNRMpNyicFNYW2fbK_WjsoF77qB9_XA&export=download", "mal-iodine.csv")
]

# Create a directory to save the files
if not os.path.exists("DoHBrw-2020"):
    os.makedirs("DoHBrw-2020")

# Loop through the file URLs and download files if not already present
for url, filename in file_urls:
    file_path = os.path.join("DoHBrw-2020", filename)
    if not os.path.exists(file_path):
        try:
            print(f"Downloading {filename}...")
            response = requests.get(url, stream=True)
            with open(file_path, "wb") as file:
                for chunk in response.iter_content(chunk_size=8192):
                    file.write(chunk)
            print(f"{filename} downloaded successfully!")
        except Exception as e:
            print(f"Error downloading {filename}: {e}")
    else:
        print(f"{filename} already exists!")



In [ ]:
# Importing required libraries
import json  # For working with JSON data
import math  # For mathematical operations
from collections import Counter  # For counting elements in a list
from os.path import join  # For joining file paths
import numpy as np  # For numerical operations and arrays
import pandas as pd  # For data manipulation and analysis
import plotly.express as px  # For interactive plotting
import plotly.figure_factory as ff  # For creating various types of figures
import plotly.graph_objects as go  # For creating customized plots
import random  # For generating random values
from tqdm.notebook import tqdm, trange  # For displaying progress bars in Jupyter Notebook
from deap import base, creator, tools, algorithms  # For evolutionary algorithms
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, f1_score  # For model evaluation metrics
from sklearn.model_selection import cross_val_score, train_test_split  # For cross-validation and data splitting
from sklearn.preprocessing import StandardScaler, LabelEncoder  # For data preprocessing
from sklearn.ensemble import RandomForestClassifier  # For building a Random Forest classifier
from sklearn.impute import SimpleImputer  # For imputing missing values
from sklearn.inspection import permutation_importance  # For feature importance analysis

from plotly.offline import iplot  # For offline plotting

# Additional imports
import matplotlib.pyplot as plt  # For creating traditional plots

# Shuffle data
from sklearn.utils import shuffle  # For shuffling data

import time

from sklearn.metrics import accuracy_score


import pickle  # Add this import statement



In [ ]:
# Read the first benign CSV file into a DataFrame called df1_benign
df1_benign = pd.read_csv('DoHBrw-2020/benign-chrome.csv', delimiter=',')

# Read the second benign CSV file into another DataFrame called df2_benign
df2_benign = pd.read_csv('DoHBrw-2020/benign-firefox.csv', delimiter=',')

# Append the contents of df2_benign to df1_benign (Note: This does not modify df1_benign in-place, it returns a new DataFrame)
df_benign = pd.concat([df1_benign, df2_benign], ignore_index=True)

# Add a new column 'DoH' to df1_benign and set all values in that column to 0, indicating benign traffic
df_benign['DoH'] = 0  # 'DoH' stands for DNS-over-HTTPS, and 0 indicates benign traffic

# Rename the column 'DoH' to 'labels' in df1_benign
df_benign = df_benign.rename(columns={'DoH': 'labels'})
df_benign


In [ ]:

# Read the first malicious CSV file into a DataFrame called df1_malic
df1_malic = pd.read_csv('DoHBrw-2020/mal-iodine.csv', delimiter=',')

# Add a new column 'DoH' to df1_malic and set all values in that column to 1, indicating malicious traffic of type 'iodine'
df1_malic['DoH'] = 1  # 1 stands for 'iodine' (a type of malicious traffic)

# Read the second malicious CSV file into another DataFrame called df2_malic
df2_malic = pd.read_csv('DoHBrw-2020/mal-dns2tcp.csv', delimiter=',')

# Add a new column 'DoH' to df2_malic and set all values in that column to 2, indicating malicious traffic of type 'dns2tcp'
df2_malic['DoH'] = 2  # 2 stands for 'dns2tcp' (another type of malicious traffic)

# Read the third malicious CSV file into another DataFrame called df3_malic
df3_malic = pd.read_csv('DoHBrw-2020/mal-dnscat2.csv', delimiter=',')

# Add a new column 'DoH' to df3_malic and set all values in that column to 3, indicating malicious traffic of type 'dnscat2'
df3_malic['DoH'] = 3  # 3 stands for 'dnscat2' (yet another type of malicious traffic)

# Concatenate the DataFrames df1_malic, df2_malic, and df3_malic into a single DataFrame
# The 'ignore_index=True' ensures that the index is reset after concatenation to avoid index duplication
df1_malic = pd.concat([df1_malic, df2_malic, df3_malic], ignore_index=True)

# Rename the column 'DoH' to 'labels' in df1_malic to have a common label indicating the type of traffic (0 for benign, 1, 2, 3 for malicious types)
df1_malic = df1_malic.rename(columns={'DoH': 'labels'})
df1_malic


In [ ]:
# Shuffle the DataFrame
data = shuffle(pd.concat([df_benign, df1_malic], ignore_index=True))

# Check the number of null (missing) values in each column of the DataFrame 'data'
null_value_counts = data.isnull().sum()

# Drop columns with the same value across all rows
columns_to_drop = [col for col in data.columns if data[col].nunique() == 1]
data_dropped = data.drop(columns=columns_to_drop)

# Fill missing values or NaN values with 0 for all columns
data_filled = data_dropped.fillna(0)

# Print the number of null values after filling
print("Null Value Counts after Filling:")
print(data_filled.isnull().sum())

# Now 'data_filled' contains the DataFrame with missing values filled with 0


In [ ]:
data

In [ ]:
# Compute the statistical summary of numeric columns in the DataFrame 'data'
data.describe()


In [ ]:
"""
The code data['SourceIP'] is used to access the 'SourceIP' column in the DataFrame data.
It retrieves the values of the 'SourceIP' column, which represents the source IP addresses of
the network traffic data.
"""
data['SourceIP']

In [ ]:
# Compute the count of each unique value in the 'labels' column of the DataFrame 'data'
data.labels.value_counts()

In [ ]:

# Map the numeric labels to their corresponding descriptions
attack_descriptions = {
    0: "Benign",
    1: "Malicious - Iodine",
    2: "Malicious - DNS2TCP",
    3: "Malicious - Dnscat2",
}

# Convert the 'TimeStamp' column to datetime if it's not already in datetime format
data['TimeStamp'] = pd.to_datetime(data['TimeStamp'])

# Group the data by 'TimeStamp' and 'labels' to get the count of each attack type at each timestamp
grouped_data = data.groupby(['TimeStamp', 'labels']).size().reset_index(name='count')

# Create a new column 'AttackTypeDescription' by mapping the 'labels' to their corresponding descriptions
grouped_data['AttackTypeDescription'] = grouped_data['labels'].map(attack_descriptions)

In [ ]:
# Create the plot
fig = px.line(
    grouped_data,
    x='TimeStamp',
    y='count',
    color='AttackTypeDescription',
    markers=True,
    hover_data={'AttackTypeDescription': True},  # Show attack descriptions on hover
)

# Update the layout for better readability (optional)
fig.update_layout(
    title='Attack Type Distribution Over Time',
    xaxis_title='Time',
    yaxis_title='Count',
    legend_title='Attack Type',
)

# Show the plot
fig.show()

In [ ]:
# Create an instance of LabelEncoder
le = LabelEncoder()

# Iterate over all columns in the DataFrame
for column in data.columns:
    # Check if the column is non-numeric (categorical)
    if data[column].dtype == 'object':
        # Fit and transform the column using LabelEncoder
        data[column] = le.fit_transform(data[column])

# Now, the non-numeric columns have been converted to numerical labels
data


In [ ]:
data['SourceIP']

In [ ]:
data.describe()

In [ ]:
# Separate the data into different classes
benign_data = data[data['labels'] == 0].head(100)
malicious_data = data[data['labels'] != 0].head(300)

# Combine the data samples
small_sample = pd.concat([benign_data, malicious_data], ignore_index=True)

# Print the small sample
# data = small_sample.copy()

In [ ]:
# Create the feature variables (X) by dropping the "TimeStamp" and "labels" columns from the DataFrame 'data'
#X = data.drop(["TimeStamp", "labels"], axis=1)
X = data.drop(["TimeStamp", "labels"], axis=1)
# 'data.drop(["TimeStamp", "labels"], axis=1)' removes the "TimeStamp" and "labels" columns from 'data' and returns a new DataFrame 'X'
# The 'axis=1' parameter specifies that we want to drop columns, not rows.

# Create the target variable (y) by extracting the values from the "labels" column of the DataFrame 'data'
#y = data['labels'].values
y = data['labels'].values

# 'data['labels']' accesses the "labels" column in 'data', and '.values' extracts the values as a NumPy array.
# The resulting 'y' will be a one-dimensional NumPy array containing the target labels.


In [ ]:

imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)


In [ ]:
# Split the data into training and testing sets using a test size of 50% (0.5) of the entire dataset
# The random_state parameter ensures reproducibility by fixing the random seed used for the split.
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.5, random_state=1)

# Further split the training set into training and validation sets using a test size of 25% (0.25) of the training set
# The random_state parameter ensures consistency between different runs by using the same random seed as before.
# The validation set size will be 25% of 50% (0.25 x 0.5 = 0.125) of the entire dataset.
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)


In [ ]:

def displayClasificationResults(z, y_test, y_pred, numClasses=4):
    # Calculate and display the number of mislabeled points and accuracy
    print("Number of mislabeled points out of a total %d points: %d"
          % (y_test.shape[0], (y_test != y_pred).sum()))
    accuracy = round(100 - (((y_test != y_pred).sum() / y_test.shape[0]) * 100), 2)
    print(f"Accuracy is {accuracy}%")

    # Calculate and display precision, recall, and F-score (weighted average)
    precision, recall, fscore, support = precision_recall_fscore_support(
        y_test, y_pred, average='weighted')
    precision *= 100
    recall *= 100
    fscore *= 100
    print(f"Precision = {round(precision, 2)}%")
    print(f"Recall = {round(recall, 2)}%")
    print(f"F-score = {round(fscore, 2)}%")

    # Set the labels for x and y axes in the confusion matrix
    if numClasses == 2:
        x = ['benign', 'malicious']
        y = ['benign', 'malicious']
    else:
        x = ['benign', 'iodine', 'dns2tcp', 'dnscat2']
        y = ['benign', 'iodine', 'dns2tcp', 'dnscat2']

    # Change each element of z to type string for annotations in the heatmap
    z_text = [[str(y) for y in x] for x in z]

    # Create an annotated heatmap using Plotly with the confusion matrix
    fig = ff.create_annotated_heatmap(
        z, x=x, y=y, annotation_text=z_text, colorscale='Viridis')

    # Add title and custom axis titles to the heatmap
    fig.update_layout(title_text='<i><b>Confusion matrix</b></i>')

    # Add custom x-axis title
    fig.add_annotation(dict(font=dict(color="black", size=14),
                            x=0.5,
                            y=-0.15,
                            showarrow=False,
                            text="Predicted value",
                            xref="paper",
                            yref="paper"))

    # Add custom y-axis title with angle adjustment
    fig.add_annotation(dict(font=dict(color="black", size=14),
                            x=-0.35,
                            y=0.5,
                            showarrow=False,
                            text="Real value",
                            textangle=-90,
                            xref="paper",
                            yref="paper"))

    # Adjust margins to make room for the y-axis title
    fig.update_layout(margin=dict(t=50, l=200))

    # Add colorbar to the heatmap
    fig['data'][0]['showscale'] = True

    # Show the heatmap
    iplot(fig)




In [ ]:
# Create a RandomForestClassifier with 500 estimators and a fixed random state for reproducibility
# rfc_4_classification = RandomForestClassifier(n_estimators=500, random_state=1)

from sklearn.ensemble import RandomForestClassifier

rfc_4_classification = RandomForestClassifier(
    n_estimators=50,
    max_depth=10,
    max_features='sqrt',
    n_jobs=-1,
    random_state=42
)

In [ ]:
# Train the RandomForestClassifier on the training data (X_train, y_train) and make predictions on the training data
y_pred = rfc_4_classification.fit(X_train, y_train).predict(X_train)

# Compute the confusion matrix using the actual training labels (y_train) and the predicted labels (y_pred)
z = confusion_matrix(y_train, y_pred)

In [ ]:
# Display the classification results using the 'displayClasificationResults' function
# The function will show the number of mislabeled points, accuracy, precision, recall, and an annotated heatmap of the confusion matrix.
displayClasificationResults(z, y_train, y_pred)

In [ ]:
# Make predictions on the test data (X_test) using the trained RandomForestClassifier
y_pred = rfc_4_classification.predict(X_test)

# Compute the confusion matrix using the actual test labels (y_test) and the predicted labels (y_pred)
z = confusion_matrix(y_test, y_pred)

# Display the classification results using the 'displayClasificationResults' function
# The function will show the number of mislabeled points, accuracy, precision, recall, and an annotated heatmap of the confusion matrix.
displayClasificationResults(z, y_test, y_pred)


In [ ]:
# Make predictions on the validation data (X_val) using the trained RandomForestClassifier
y_pred = rfc_4_classification.predict(X_val)

# Compute the confusion matrix using the actual validation labels (y_val) and the predicted labels (y_pred)
z = confusion_matrix(y_val, y_pred)

# Display the classification results using the 'displayClasificationResults' function
# The function will show the number of mislabeled points, accuracy, precision, recall, and an annotated heatmap of the confusion matrix.
displayClasificationResults(z, y_val, y_pred)


In [ ]:


# Calculate permutation feature importance
perm_importance = permutation_importance(rfc_4_classification, X_test, y_test, n_repeats=30, random_state=1)

# Obtain feature names
feature_names = list(X.columns)

# Sort features by importance scores
sorted_idx = perm_importance.importances_mean.argsort()

# Plot feature importance
plt.figure(figsize=(10, 6))
plt.barh(range(len(sorted_idx)), perm_importance.importances_mean[sorted_idx], align='center')
plt.yticks(range(len(sorted_idx)), [feature_names[i] for i in sorted_idx])
plt.xlabel('Permutation Importance')
plt.title('Feature Importance - Permutation Importance')
plt.show()
# Print feature names and their importance values in descending order
for idx in reversed(sorted_idx):
    print(f"{feature_names[idx]}: {perm_importance.importances_mean[idx]}")


In [ ]:
# Print feature names and their importance values in descending order
for idx in reversed(sorted_idx):
    print(f"{feature_names[idx]}: {perm_importance.importances_mean[idx]}")


In [ ]:
# Extract the top five feature names and their importance values
top_feature_indices = sorted_idx[-5:]
top_feature_names = [feature_names[idx] for idx in top_feature_indices]
top_feature_importances = [perm_importance.importances_mean[idx] for idx in top_feature_indices]

# Print the top five features and their importance values
print("Top Five Features and Their Importance Values:")
for feature, importance in zip(top_feature_names, top_feature_importances):
    print(f"{feature}: {importance}")

In [ ]:
# Create a deep copy of data and name it newdata
newdata = data.copy()

# Create new combinations of the top five features using the mean
new_feature_combinations = []
for i in range(5):
    for j in range(i + 1, 5):
        new_combination = f"{top_feature_names[i]}_{top_feature_names[j]}_mean"
        new_feature_combinations.append(new_combination)

# Add the new feature combinations (mean) to the newdata dataframe
for combination in new_feature_combinations:
    feature_indices = [top_feature_names.index(name) for name in combination.split('_')[:-1]]
    newdata[combination] = newdata[top_feature_names].iloc[:, feature_indices].mean(axis=1)


# Print the first few rows of the dataframe to verify the additions
print("Updated DataFrame with New Feature Combinations:")
newdata


In [ ]:
# Prepare the new features and labels for machine learning
X_new = newdata[new_feature_combinations].values
y_new = newdata['labels'].values


# Split the data into training, validation, and testing sets
X_train_all, X_temp_all, y_train_all, y_temp_all = train_test_split(X_new, y_new, test_size=0.5, random_state=1)
X_val_all, X_test_all, y_val_all, y_test_all = train_test_split(X_temp_all, y_temp_all, test_size=0.25, random_state=1)

# Create a new RandomForestClassifier for the updated dataset
# rfc_new_all = RandomForestClassifier(n_estimators=500, random_state=1)
rfc_new_all = RandomForestClassifier(
    n_estimators=50,
    max_depth=10,
    max_features='sqrt',
    n_jobs=-1,
    random_state=42
)
# Train the new classifier on the training data
rfc_new_all.fit(X_train_all, y_train_all)

# Make predictions on the training, validation, and testing data
y_pred_train_all = rfc_new_all.predict(X_train_all)
y_pred_val_all = rfc_new_all.predict(X_val_all)
y_pred_test_all = rfc_new_all.predict(X_test_all)


In [ ]:
# Display classification results for the training data
print("Classification Results for Training Data:")
displayClasificationResults(confusion_matrix(y_train_all, y_pred_train_all), y_train_all, y_pred_train_all)

In [ ]:
# Display classification results for the validation data
print("Classification Results for Validation Data:")
displayClasificationResults(confusion_matrix(y_val_all, y_pred_val_all), y_val_all, y_pred_val_all)

In [ ]:
# Display classification results for the testing data
print("Classification Results for Testing Data:")
displayClasificationResults(confusion_matrix(y_test_all, y_pred_test_all), y_test_all, y_pred_test_all)

In [ ]:
# Create the feature variables (X) by dropping the "TimeStamp" and "labels" columns from the DataFrame 'data'
X_all = newdata.drop(["TimeStamp", "labels"], axis=1)

# Create the target variable (y) by extracting the values from the "labels" column of the DataFrame 'data'
y_all = newdata['labels'].values
X_imputed_all = imputer.fit_transform(X_all)
X_scaled_all = scaler.fit_transform(X_imputed_all)


# Split the data into training, validation, and testing sets
X_train_all, X_temp_all, y_train_all, y_temp_all = train_test_split(X_scaled_all, y_all, test_size=0.5, random_state=1)
X_val_all, X_test_all, y_val_all, y_test_all = train_test_split(X_temp_all, y_temp_all, test_size=0.25, random_state=1)

# Create a new RandomForestClassifier for the updated dataset
# rfc_new_all = RandomForestClassifier(n_estimators=500, random_state=1)
rfc_new_all = RandomForestClassifier(
    n_estimators=50,
    max_depth=10,
    max_features='sqrt',
    n_jobs=-1,
    random_state=42
)
# Train the new classifier on the training data
rfc_new_all.fit(X_train_all, y_train_all)

# Make predictions on the training, validation, and testing data
y_pred_train_all = rfc_new_all.predict(X_train_all)
y_pred_val_all = rfc_new_all.predict(X_val_all)
y_pred_test_all = rfc_new_all.predict(X_test_all)


In [ ]:
# Display classification results for the training data
print("Classification Results for Training Data:")
displayClasificationResults(confusion_matrix(y_train_all, y_pred_train_all), y_train_all, y_pred_train_all)

In [ ]:
# Display classification results for the validation data
print("Classification Results for Validation Data:")
displayClasificationResults(confusion_matrix(y_val_all, y_pred_val_all), y_val_all, y_pred_val_all)

In [ ]:
# Display classification results for the testing data
print("Classification Results for Testing Data:")
displayClasificationResults(confusion_matrix(y_test_all, y_pred_test_all), y_test_all, y_pred_test_all)

In [ ]:

# Calculate permutation feature importance
perm_importance = permutation_importance(rfc_new_all, X_test_all, y_test_all, n_repeats=30, random_state=1)

# Obtain feature names
feature_names = list(X_all.columns)

# Sort features by importance scores
sorted_idx = perm_importance.importances_mean.argsort()

# Plot feature importance
plt.figure(figsize=(10, 6))
plt.barh(range(len(sorted_idx)), perm_importance.importances_mean[sorted_idx], align='center')
plt.yticks(range(len(sorted_idx)), [feature_names[i] for i in sorted_idx])
plt.xlabel('Permutation Importance')
plt.title('Feature Importance - Permutation Importance')
plt.show()
# Print feature names and their importance values in descending order
for idx in reversed(sorted_idx):
    print(f"{feature_names[idx]}: {perm_importance.importances_mean[idx]}")

In [ ]:
# =======================================================================================
#
#  Enhanced DNS Tunneling Detection: Feature Selection using Metaheuristics
#
#  Author:      [Mahmoud Sammour / UTeM]
#  Date:        August 4, 2025
#  Version:     2.1 (Modified for comparative visualization)
#
#  Project:
#  This script applies and compares two advanced metaheuristic algorithms,
#  RLGWO and its enhanced version, to perform feature selection for a
#  DNS tunneling detection model. The goal is to identify a minimal, yet highly
#  predictive, subset of features to improve model efficiency and performance.
#
#  Methodology:
#  This project uses a public dataset for DNS-over-HTTPS (DoH) traffic analysis,
#  combining benign Browse data with three types of malicious DNS tunneling
#  traffic. This creates a multi-class classification problem with a significant
#  class imbalance, justifying the use of a weighted F1-score for evaluation.
#  - Classes: Benign, Iodine, DNS2TCP, Dnscat2.
#
#  The dataset, containing approximately 1.17 million samples, is divided into
#  three sets for robust model development and evaluation:
#  - Training Set:   50%   (~584k samples)
#  - Validation Set: 37.5% (~438k samples)
#  - Test Set:       12.5% (~146k samples)
#
#  A two-phase approach is used to balance optimization speed and solution quality:
#  1.  Optimization Phase: A fast, lightweight Random Forest model is used as a
#      fitness proxy to allow the optimizers to explore the vast search space
#      of feature combinations quickly.
#  2.  Validation Phase: The best feature set discovered by each algorithm is then
#      validated using a full-sized, robust Random Forest model to determine its
#      true performance.
#
# =======================================================================================

# --- Core Libraries ---
import time
import pickle
import numpy as np
import random
from collections import defaultdict

# --- Machine Learning & Metrics ---
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, classification_report, confusion_matrix

# --- Deep Learning (PyTorch) ---
import torch
import torch.nn as nn
import torch.optim as optim

# --- Visualization ---
from tqdm.notebook import tqdm
import plotly.graph_objects as go
import plotly.figure_factory as ff


# =======================================================================================
# --- Section 1: Fitness Evaluation & Binary Adaptation Helpers ---
# =======================================================================================

def evaluate_features_fast(individual):
    """
    Calculates the fitness of a feature set for the optimizers.

    This function serves as a fast "proxy" for model performance. It uses a
    lightweight Random Forest model to quickly evaluate a given solution. The
    fitness score is a trade-off between the model's predictive power (F1-score)
    and its complexity (number of features), promoting simpler, more generalizable
    models.

    Args:
        individual (list): A binary list where a `1` at index `i` indicates that
                           feature `i` is selected for evaluation.

    Returns:
        tuple: A tuple containing a single float value representing the fitness.
               The comma is required for compatibility with some optimization libraries.
    """
    selected_indices = [i for i, selected in enumerate(individual) if selected]
    if not selected_indices:
        return 0.0,  # Return the worst possible fitness if no features are selected.

    # Slice the global dataset variables to use only the selected features.
    X_train_subset = X_train_all[:, selected_indices]
    X_test_subset = X_test_all[:, selected_indices]

    # Define a lightweight classifier. Key parameters (n_estimators, max_depth) are
    # kept low to ensure this evaluation step is fast.
    classifier = RandomForestClassifier(n_estimators=20, max_depth=8, random_state=42, n_jobs=-1)
    classifier.fit(X_train_subset, y_train_all)

    y_pred = classifier.predict(X_test_subset)
    score = f1_score(y_test_all, y_pred, average='weighted')

    # Apply a penalty for model complexity. This is the correct penalty for this problem.
    penalty = 0.05 * (sum(individual) / len(individual))

    return score - penalty,


def _binarize_wolf(continuous_position):
    """
    Converts a continuous position vector from GWO into a binary vector.

    This is the core adaptation that allows a continuous optimizer like GWO to solve
    a binary problem. It uses a sigmoid transfer function to map the agent's
    continuous position to a vector of probabilities. A stochastic threshold then
    converts these probabilities into a definitive binary choice (0 or 1) for each feature.

    Args:
        continuous_position (np.ndarray): The GWO agent's position in continuous space.

    Returns:
        list: The corresponding binary position vector for feature selection.
    """
    # Sigmoid function squashes any real number into a (0, 1) probability.
    probabilities = 1 / (1 + np.exp(-np.array(continuous_position)))
    
    # For each feature, if a random number is less than its probability, select it (1).
    binary_position = (np.random.random(len(probabilities)) < probabilities).astype(int)

    # A solution with zero features is invalid. If this occurs, randomly flip one
    # gene to 1 to ensure validity.
    if np.sum(binary_position) == 0:
        binary_position[np.random.randint(0, len(binary_position))] = 1

    return binary_position.tolist()


def binary_opposition_based_learning(population):
    """
    Generates the "opposite" of a binary population by flipping all bits.

    This strategy is used as a powerful exploration mechanism, allowing the optimizer
    to escape local optima by making a large jump to a completely different region
    of the search space.

    Args:
        population (list of lists): The current population of binary individuals.

    Returns:
        list of lists: A new population where each individual is the bitwise
                       complement of an individual in the original population.
    """
    return [[1 - bit for bit in individual] for individual in population]


# =======================================================================================
# --- Section 2: Deep Reinforcement Learning Architecture ---
# =======================================================================================

class DuelingDQN(nn.Module):
    """
    Implements a Dueling Deep Q-Network (DQN) architecture.

    A Dueling DQN has two streams to separately estimate state values (V(s)) and
    action advantages (A(s,a)). This separation allows the network to learn which
    states are (or are not) valuable without having to learn the effect of each
    action for each state. This is particularly useful in environments where actions
    do not affect the environment in any relevant way.

    Reference:
        Wang, Z., Schaul, T., Hessel, M., et al. (2016). "Dueling Network
        Architectures for Deep Reinforcement Learning." PMLR.
    """
    def __init__(self, state_size, action_size):
        super(DuelingDQN, self).__init__()
        self.feature_layer = nn.Sequential(nn.Linear(state_size, 64), nn.ReLU())
        
        # The Value Stream: predicts V(s) - how good is it to be in this state?
        self.value_stream = nn.Sequential(nn.Linear(64, 64), nn.ReLU(), nn.Linear(64, 1))
        
        # The Advantage Stream: predicts A(s,a) - how much better is taking this
        # action compared to the other possible actions?
        self.advantage_stream = nn.Sequential(nn.Linear(64, 64), nn.ReLU(), nn.Linear(64, action_size))

    def forward(self, state):
        features = self.feature_layer(state)
        value = self.value_stream(features)
        advantages = self.advantage_stream(features)
        
        # Combine value and advantages to get the final Q-values. The mean of the
        # advantages is subtracted to ensure identifiability.
        return value + (advantages - advantages.mean(dim=1, keepdim=True))


# =======================================================================================
# --- Section 3: Optimizer Algorithm Implementations ---
# =======================================================================================

class BinaryRLGWO:
    """
    A Reinforcement Learning-Guided Grey Wolf Optimizer (RLGWO) adapted for
    binary feature selection problems. It uses a simple Q-table to learn a
    policy for balancing exploration and exploitation.
    """
    def __init__(self, n_dimensions, population_size, generations):
        self.n_dimensions, self.population_size, self.generations = n_dimensions, population_size, generations
        self.actions = ["exploration", "exploitation"]
        self.q_table = defaultdict(lambda: 0)
        self.epsilon, self.alpha, self.gamma = 0.2, 0.1, 0.9

    def choose_action(self, state):
        if random.random() < self.epsilon: return random.choice(self.actions)
        return self.actions[np.argmax([self.q_table[(state, action)] for action in self.actions])]

    def update_q_table(self, state, action, reward, next_state):
        max_next_q = max(self.q_table.get((next_state, a), 0) for a in self.actions)
        self.q_table[(state, action)] += self.alpha * (reward + self.gamma * max_next_q - self.q_table[(state, action)])

    def run(self, evaluate_func):
        population = [[random.randint(0, 1) for _ in range(self.n_dimensions)] for _ in range(self.population_size)]
        fitnesses = [evaluate_func(ind)[0] for ind in population]
        state = f"{max(fitnesses):.3f}_{np.std(fitnesses):.3f}"
        best_individual_so_far = population[np.argmax(fitnesses)]
        best_fitness_so_far = max(fitnesses)
        history = [best_fitness_so_far]
        progress_bar = tqdm(range(1, self.generations + 1), desc="Standard RLGWO")
        for gen in progress_bar:
            action = self.choose_action(state)
            a_factor = 1.5 if action == "exploration" else 0.5
            pop_with_fitness = sorted(zip(population, fitnesses), key=lambda x: x[1], reverse=True)
            alpha, beta, delta = pop_with_fitness[0][0], pop_with_fitness[1][0], pop_with_fitness[2][0]
            new_population = []
            for i in range(self.population_size):
                A1,A2,A3 = a_factor*(2*np.random.rand(self.n_dimensions)-1), a_factor*(2*np.random.rand(self.n_dimensions)-1), a_factor*(2*np.random.rand(self.n_dimensions)-1)
                C1,C2,C3 = 2*np.random.rand(self.n_dimensions), 2*np.random.rand(self.n_dimensions), 2*np.random.rand(self.n_dimensions)
                D_alpha, D_beta, D_delta = np.abs(C1*np.array(alpha) - population[i]), np.abs(C2*np.array(beta) - population[i]), np.abs(C3*np.array(delta) - population[i])
                X1,X2,X3 = np.array(alpha)-A1*D_alpha, np.array(beta)-A2*D_beta, np.array(delta)-A3*D_delta
                new_population.append(_binarize_wolf((X1 + X2 + X3) / 3.0))
            population = new_population; fitnesses = [evaluate_func(ind)[0] for ind in population]
            if max(fitnesses) > best_fitness_so_far:
                best_fitness_so_far = max(fitnesses); best_individual_so_far = population[np.argmax(fitnesses)]
            history.append(best_fitness_so_far)
            reward = max(fitnesses) + 0.1 * np.std(fitnesses)
            next_state = f"{max(fitnesses):.3f}_{np.std(fitnesses):.3f}"
            self.update_q_table(state, action, reward, next_state); state = next_state
            progress_bar.set_postfix(best_fitness=f"{best_fitness_so_far:.4f}", features=f"{sum(best_individual_so_far)}")
        return best_individual_so_far, best_fitness_so_far, history

class EnhancedBinaryRLGWO:
    """
    An enhanced RLGWO adapted for binary problems, featuring a Dueling DQN,
    Prioritized Experience Replay (PER), and Opposition-Based Learning (OBL).
    This represents the full, non-simplified version of the algorithm.
    """
    def __init__(self, n_dimensions, population_size, generations):
        self.n_dimensions = n_dimensions
        self.population_size = population_size
        self.generations = generations
        # --- Algorithm Actions ---
        # The agent can choose to modulate the GWO 'a' parameter for exploration/exploitation
        # or trigger a strategic jump with Opposition-Based Learning.
        self.actions = [("set_a", 0.5), ("set_a", 1.0), ("set_a", 1.5), ("activate_obl", None)]
        # --- RL Hyperparameters ---
        self.epsilon, self.epsilon_end, self.epsilon_decay = 1.0, 0.01, 0.995
        self.gamma, self.tau, self.batch_size = 0.99, 0.005, 64
        # --- PER Hyperparameters ---
        self.replay_buffer_size, self.beta, self.priority_epsilon = 2000, 0.4, 1e-6
        self.replay_buffer = []
        # --- PyTorch Setup ---
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.q_network = DuelingDQN(3, len(self.actions)).to(self.device)
        self.target_q_network = DuelingDQN(3, len(self.actions)).to(self.device)
        self.target_q_network.load_state_dict(self.q_network.state_dict())
        self.optimizer = optim.Adam(self.q_network.parameters(), lr=0.001)
    
    def add_to_buffer(self, experience):
        max_priority = max([p for p, _ in self.replay_buffer]) if self.replay_buffer else 1.0
        self.replay_buffer.append((max_priority, experience))
        if len(self.replay_buffer) > self.replay_buffer_size: self.replay_buffer.pop(0)

    ### NON-SIMPLIFIED: Full Prioritized Experience Replay (PER) ###
    def replay(self):
        """
        Trains the DQN using Prioritized Experience Replay (PER), which samples
        more important transitions more frequently.

        Reference:
            Schaul, T., Quan, J., Antonoglou, I., & Silver, D. (2015).
            "Prioritized Experience Replay." arXiv.
        """
        if len(self.replay_buffer) < self.batch_size: return
        
        priorities = np.array([p for p, _ in self.replay_buffer]); probs = priorities ** self.beta; probs /= probs.sum()
        indices = np.random.choice(len(self.replay_buffer), self.batch_size, p=probs)
        batch = [self.replay_buffer[i][1] for i in indices]
        
        weights = (len(self.replay_buffer) * probs[indices]) ** (-self.beta); weights /= weights.max()
        weights = torch.tensor(weights, dtype=torch.float32).to(self.device)
        
        states, actions, rewards, next_states = zip(*batch)
        states = torch.tensor(np.array(states), dtype=torch.float32).to(self.device)
        next_states = torch.tensor(np.array(next_states), dtype=torch.float32).to(self.device)
        rewards = torch.tensor(rewards, dtype=torch.float32).to(self.device)
        action_indices = torch.tensor(actions, dtype=torch.int64).to(self.device)
        
        predicted_q = self.q_network(states).gather(1, action_indices.unsqueeze(1)).squeeze(1)
        with torch.no_grad(): next_q_values = self.target_q_network(next_states).max(1)[0]
        target_q = rewards + self.gamma * next_q_values
        td_errors = torch.abs(target_q - predicted_q).detach()
        loss = (weights * nn.MSELoss(reduction='none')(predicted_q, target_q)).mean()
        
        self.optimizer.zero_grad(); loss.backward(); self.optimizer.step()
        
        for i, idx in enumerate(indices):
            self.replay_buffer[idx] = (td_errors[i].item() + self.priority_epsilon, self.replay_buffer[idx][1])
            
        for target_param, local_param in zip(self.target_q_network.parameters(), self.q_network.parameters()):
            target_param.data.copy_(self.tau * local_param.data + (1.0 - self.tau) * target_param.data)

    def choose_action_index(self, state):
        if random.random() < self.epsilon: return random.randrange(len(self.actions))
        state_tensor = torch.tensor(state, dtype=torch.float32).to(self.device).unsqueeze(0)
        with torch.no_grad(): q_values = self.q_network(state_tensor)
        return torch.argmax(q_values).item()

    def run(self, evaluate_func):
        """Executes the optimization process."""
        population = [[random.randint(0, 1) for _ in range(self.n_dimensions)] for _ in range(self.population_size)]
        fitnesses = [evaluate_func(ind)[0] for ind in population]
        state = [max(fitnesses), np.mean(fitnesses), np.std(fitnesses)]
        best_individual_so_far = population[np.argmax(fitnesses)]; best_fitness_so_far = max(fitnesses)
        history = [best_fitness_so_far]

        progress_bar = tqdm(range(1, self.generations + 1), desc="Enhanced RLGWO")
        for gen in progress_bar:
            self.beta = min(1.0, self.beta + 0.001)
            action_index = self.choose_action_index(state)
            action_type, action_value = self.actions[action_index]
            
            if action_type == "activate_obl":
                opposite_pop = binary_opposition_based_learning(population); combined_pop = population + opposite_pop
                combined_fitnesses = [evaluate_func(ind)[0] for ind in combined_pop]
                sorted_combined = sorted(zip(combined_pop, combined_fitnesses), key=lambda x: x[1], reverse=True)
                population = [ind for ind, fit in sorted_combined[:self.population_size]]
                fitnesses = [fit for ind, fit in sorted_combined[:self.population_size]]
            else: # "set_a"
                a_factor = action_value
                pop_with_fitness = sorted(zip(population, fitnesses), key=lambda x: x[1], reverse=True)
                alpha, beta, delta = pop_with_fitness[0][0], pop_with_fitness[1][0], pop_with_fitness[2][0]
                new_population = []
                for i in range(self.population_size):
                    A1,A2,A3 = a_factor*(2*np.random.rand(self.n_dimensions)-1),a_factor*(2*np.random.rand(self.n_dimensions)-1),a_factor*(2*np.random.rand(self.n_dimensions)-1)
                    C1,C2,C3 = 2*np.random.rand(self.n_dimensions), 2*np.random.rand(self.n_dimensions), 2*np.random.rand(self.n_dimensions)
                    D_alpha=np.abs(C1*np.array(alpha)-population[i]); D_beta=np.abs(C2*np.array(beta)-population[i]); D_delta=np.abs(C3*np.array(delta)-population[i])
                    X1=np.array(alpha)-A1*D_alpha; X2=np.array(beta)-A2*D_beta; X3=np.array(delta)-A3*D_delta
                    new_population.append(_binarize_wolf((X1 + X2 + X3) / 3.0))
                population = new_population; fitnesses = [evaluate_func(ind)[0] for ind in population]
            
            if max(fitnesses) > best_fitness_so_far:
                best_fitness_so_far = max(fitnesses); best_individual_so_far = population[np.argmax(fitnesses)]
            history.append(best_fitness_so_far)
            
            next_state = [max(fitnesses), np.mean(fitnesses), np.std(fitnesses)]
            reward = next_state[0] + 0.1 * next_state[2]
            self.add_to_buffer((state, action_index, reward, next_state)); self.replay(); state = next_state
            self.epsilon = max(self.epsilon_end, self.epsilon_decay * self.epsilon)
            progress_bar.set_postfix(best_fitness=f"{best_fitness_so_far:.4f}", features=f"{sum(best_individual_so_far)}")
        
        return best_individual_so_far, best_fitness_so_far, history

# =======================================================================================
# --- Section 4: Main Experiment Execution & Visualization ---
# =======================================================================================

### --- Phase 1: Run Fast Optimization --- ###
print("--- Phase 1: Running Fast Optimization ---")
pop_size = 40
generations = 15
n_features = X_train_all.shape[1]

# Assume feature_names is a list of strings corresponding to the columns of X_train_all
# e.g., feature_names = ['feature_1', 'feature_2', ...]

gwo = BinaryRLGWO(n_features, pop_size, generations)
best_sol_gwo, _, history_gwo = gwo.run(evaluate_features_fast)

enhanced_gwo = EnhancedBinaryRLGWO(n_features, pop_size, generations)
best_sol_enhanced, _, history_enhanced = enhanced_gwo.run(evaluate_features_fast)
print("\n✅ Optimization Phase Complete.")


### --- NEW HELPER: Full Validation and Plotting Function --- ###
def validate_and_plot_results(solution_binary, algorithm_name, feature_names, X_train, y_train, X_test, y_test, class_names):
    """
    Validates a feature set, prints metrics, and plots a confusion matrix.
    """
    print("\n" + "="*60)
    print(f"🏆 VALIDATING: {algorithm_name} 🏆")
    print("="*60)

    solution_indices = [i for i, selected in enumerate(solution_binary) if selected]
    selected_feature_names = [feature_names[i] for i in solution_indices]

    if not solution_indices:
        print("No features were selected. Cannot validate model.")
        return

    print(f"Found {len(selected_feature_names)} features. Training full-sized model...")
    X_train_final = X_train[:, solution_indices]
    X_test_final = X_test[:, solution_indices]

    # Define and train the full-sized, "heavyweight" classifier.
    final_classifier = RandomForestClassifier(n_estimators=500, random_state=42, n_jobs=-1)
    final_classifier.fit(X_train_final, y_train)
    y_pred_final = final_classifier.predict(X_test_final)
    print(f"✅ {algorithm_name} Model Trained and Evaluated.")

    print("\nClassification Report:\n")
    print(classification_report(y_test, y_pred_final, target_names=class_names))

    print("\nConfusion Matrix:\n")
    cm = confusion_matrix(y_test, y_pred_final)
    cm_text = [[str(y) for y in x] for x in cm]

    fig_cm = ff.create_annotated_heatmap(
        z=cm, x=class_names, y=class_names, annotation_text=cm_text, colorscale='Purples'
    )
    fig_cm.update_layout(
        title_text=f'<b>{algorithm_name} Confusion Matrix</b>',
        xaxis_title='Predicted Label',
        yaxis_title='True Label'
    )
    fig_cm.show()

    print("\n" + "="*50)
    print(f"Features Selected by {algorithm_name} ({len(selected_feature_names)} total):")
    for name in selected_feature_names: print(f"- {name}")
    print("="*50)


### --- Phase 2: Perform Full Validation on Best Solutions --- ###
print("\n\n--- Phase 2: Performing Full Validation on Results from Each Algorithm ---")
class_names = ['Benign', 'Iodine', 'DNS2TCP', 'Dnscat2']

# Validate the solution from the Standard RLGWO
validate_and_plot_results(
    solution_binary=best_sol_gwo,
    algorithm_name="Standard RLGWO",
    feature_names=feature_names,
    X_train=X_train_all,
    y_train=y_train_all,
    X_test=X_test_all,
    y_test=y_test_all,
    class_names=class_names
)

# Validate the solution from the Enhanced RLGWO
validate_and_plot_results(
    solution_binary=best_sol_enhanced,
    algorithm_name="Enhanced RLGWO",
    feature_names=feature_names,
    X_train=X_train_all,
    y_train=y_train_all,
    X_test=X_test_all,
    y_test=y_test_all,
    class_names=class_names
)


### --- Final Convergence Plot --- ###
def plot_convergence(history_gwo, history_enhanced):
    """Plots the fitness history of both optimizers using Plotly."""
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=history_gwo, mode='lines+markers', name='Standard RLGWO', line=dict(color='royalblue')))
    fig.add_trace(go.Scatter(y=history_enhanced, mode='lines+markers', name='Enhanced RLGWO (with PER)', line=dict(color='firebrick')))
    fig.update_layout(
        title='<b>Optimizer Convergence During Fast Training</b>',
        xaxis_title='Generation',
        yaxis_title='Best Fitness (F1-Score - Penalty)',
        legend_title='Algorithm',
        template='plotly_white'
    )
    fig.show()

print("\n\n--- Plotting Optimizer Convergence ---")
plot_convergence(history_gwo, history_enhanced)